In [ ]:
import numpy as np
import scanpy as sc
import cell2location
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42

In [ ]:
import mygene
import scanpy as sc

In [ ]:
adata_vis = sc.read_visium("/root/data/LM_1")

In [ ]:
adata_vis.obs['sample'] = list(adata_vis.uns['spatial'].keys())[0]

In [ ]:
adata_vis.var['SYMBOL'] = adata_vis.var_names

In [ ]:
#adata_vis.var.set_index('gene_ids', drop=True, inplace=True)

In [ ]:
adata_vis.var

In [ ]:
adata_ref = sc.read("./data/LM_scRNA_for_C2L.h5ad")

In [ ]:
adata_ref.var['SYMBOL'] = adata_ref.var.index

In [ ]:
adata_ref.var.index

In [ ]:
# rename 'GeneID-2' as necessary for your data
#adata_ref.var.set_index('GeneID-2', drop=True, inplace=True)

# delete unnecessary raw slot (to be removed in a future version of the tutorial)
#del adata_ref.raw

In [ ]:
adata_ref.var.set_index

In [ ]:
adata_ref.var

In [ ]:
results_folder = './results/LM/'
# create paths and names to results folders for reference regression and cell2location models
ref_run_name = f'{results_folder}/reference_signatures'
run_name = f'{results_folder}/cell2location_map'

In [ ]:
adata_vis.var

In [ ]:
#去除空间转录组数据中线粒体基因的影响
adata_vis.var['MT_gene'] = [gene.startswith('MT-') for gene in adata_vis.var['SYMBOL']]
adata_vis.obsm['MT'] = adata_vis[:,adata_vis.var['MT_gene'].values].X.toarray()
adata_vis = adata_vis[:, ~adata_vis.var['MT_gene'].values]

In [ ]:
adata_ref.var_names

In [ ]:
#过滤低表达基因
from cell2location.utils.filtering import filter_genes
selected = filter_genes(adata_ref, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)
adata_ref = adata_ref[:, selected].copy()

In [ ]:
adata_ref.obs

In [ ]:
#adata_ref.obs
cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                           batch_key='orig.ident',
                           labels_key='Subset'
                           #categorical_covariate_keys=['Method']
                          )

In [ ]:
#cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                        # 10X reaction / sample / batch
                       # batch_key='Sample',
                        # cell type, covariate used for constructing signatures
                       # labels_key='Subset'
                        # multiplicative technical effects (platform, 3' vs 5', donor effect)
                        #categorical_covariate_keys=['Method']
                       #)

In [ ]:
from cell2location.models import RegressionModel
mod = RegressionModel(adata_ref)
mod.view_anndata_setup()

In [ ]:
#训练模型，要在数据上实现收敛，需要增加max_epochs参数。如果服务器有GPU，将use_gpu设置成True将显著加快训练速度。
mod.train(max_epochs=150, use_gpu=True)
mod.plot_history(20)

In [ ]:
mod.plot_history(10)
plt.legend(labels=['full data training'])

In [ ]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
#基于后验分布，计算基因在cluster中的表达
adata_ref = mod.export_posterior(
    adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500, 'use_gpu': True}
)

In [ ]:
# Save model
mod.save(f"{ref_run_name}", overwrite=True)
mod.plot_QC()

# Save anndata object with results
adata_file = f"{ref_run_name}/sc.h5ad"
#adata_ref.write(adata_file)
adata_file

In [ ]:
ref_run_name

In [ ]:
#adata_file = f"{ref_run_name}/sc.h5ad"
#adata_ref = sc.read_h5ad(adata_file)
mod = cell2location.models.RegressionModel.load(f"{ref_run_name}", adata_ref)

In [ ]:
#负二项式回归模型的所有参数都导出到参考 anndata 对象中，但是对于空间映射，我们只需要每种细胞类型中每个基因的估计表达。在这里，我们从标准输出中提取：
#因为批次的问题，计算出来的特征基因的表达和平均表达并不相同
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}' for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}' for i in adata_ref.uns['mod']['factor_names']]].copy()

inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver.iloc[0:5, 0:5]

In [ ]:
#筛选交集基因
# find shared genes and subset both anndata and reference signatures
intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)

In [ ]:
inf_aver = inf_aver.loc[intersect, :].copy()

In [ ]:
inf_aver = inf_aver[inf_aver.index.isin(intersect)].copy()

In [ ]:
#print("First few var_names in adata_vis:", adata_vis.var_names[:5])
# 检查 intersect 中的元素是否都在 adata_vis.var_names 中
#if not all(elem in adata_vis.var_names for elem in intersect):
   # print("Not all elements in 'intersect' are in 'adata_vis.var_names'.")

In [ ]:
adata_vis=  adata_vis[:, adata_vis.var_names.isin(intersect)].copy()### 鉴定一致性

In [ ]:
# 找出重复基因的位置（保留第一个出现的位置，去除后续的重复）
duplicates_to_remove = adata_vis.var_names.duplicated(keep='first')
# 仅保留非重复的基因
adata_vis = adata_vis[:, ~duplicates_to_remove].copy()
# 检查是否还存在重复
print(adata_vis.var_names.duplicated().any())

In [ ]:
adata_vis= adata_vis[:,intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()

In [ ]:
# prepare anndata for cell2location model
cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key='sample')

In [ ]:
#N_cells_per_location：每个点包含的细胞数量
#detection_alpha：如果切片间或者批次间RNA检测存在大的变异性，detection_alpha设置为20，反之可以设置为200。
# create and train the model
mod = cell2location.models.Cell2location(
    adata_vis, cell_state_df=inf_aver,
    # the expected average cell abundance: tissue-dependent
    # hyper-prior which can be estimated from paired histology:
    N_cells_per_location=30,
    # hyperparameter controlling normalisation of
    # within-experiment variation in RNA detection:
    detection_alpha=20
)
mod.view_anndata_setup()

In [ ]:
mod.train(max_epochs=3000,
          # train using full data (batch_size=None)
          batch_size=None,
          # use all data points in training because
          # we need to estimate cell abundance at all locations
          train_size=1,
          use_gpu=True,
         )

In [ ]:
# plot ELBO loss history during training, removing first 100 epochs from the plot
mod.plot_history(1000)
plt.legend(labels=['full data training']);

In [ ]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
adata_vis = mod.export_posterior(
    adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs, 'use_gpu': True}
)

# Save model
mod.save(f"{run_name}", overwrite=True)

# mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

In [ ]:
# Save anndata object with results
adata_file = f"{run_name}/sp.h5ad"
adata_vis.write(adata_file)
adata_file

In [ ]:
adata_file = f"{run_name}/sp.h5ad"
adata_vis = sc.read_h5ad(adata_file)
mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

In [ ]:
#当对多个空间批次进行整合时，以及当使用载玻片中检测到的RNA存在很大差异的数据集时（这不能用组织学中的高细胞密度来解释），
#重要的是要评估cell2location是否使这些效应归一化。您期望在不同批次之间看到相似的总细胞丰度，但不同的 RNA 检测灵敏度（均由 cell2location 估计）。您期望总细胞丰度反映组织学中的高细胞密度。

In [ ]:
# add 5% quantile, representing confident cell abundance, 'at least this amount is present',
# to adata.obs with nice names for plotting
adata_vis.obs[adata_vis.uns['mod']['factor_names']] = adata_vis.obsm['q05_cell_abundance_w_sf']

In [ ]:
adata_vis.obsm['q05_cell_abundance_w_sf']

In [ ]:
adata_vis.obs.to_csv('LM1_file.csv')

In [ ]:
#print(adata_vis.obs['batch_key'].unique())
print(adata_vis.uns["spatial"].keys())

In [ ]:
# select one slide
from cell2location.utils import select_slide
slide = select_slide(adata_vis, 'gan1')

In [ ]:
print(slide.obsm["spatial"][:5])  # 假设坐标数据存储在 obsm["spatial"]

In [ ]:
slide.obsm['spatial'] = np.array(slide.obsm['spatial'], dtype=float)

In [ ]:
# plot in spatial coordinates
with mpl.rc_context({'axes.facecolor': 'black', 'figure.figsize': [4.5, 5]}):
    sc.pl.spatial(slide, cmap='magma',
                  color=['B/Plasma', 'Endothelial', 'Epithelial', 'Fibroblasts', 'Myeloid', 'T/ILC'],
                  ncols=3, size=1.5,
                  img_key='hires',
                  vmin=0, vmax='p99.2'
                 )

In [ ]:
plt.show()

In [ ]:
print(slide.obsm["spatial"][:5])  # 假设坐标数据存储在 obsm["spatial"]

In [ ]:
# Now we use cell2location plotter that allows showing multiple cell types in one panel
from cell2location.plt import plot_spatial

# select up to 4 clusters
clust_labels = ['Fibroblasts', 'Myeloid', 'T/ILC','B/Plasma']
clust_col = ['' + str(i) for i in clust_labels] # in case column names differ from labels

slide = select_slide(adata_vis, 'gan1')
slide.obsm['spatial'] = np.array(slide.obsm['spatial'], dtype=float)


In [ ]:
cell2location.plt.plot_spatial(adata=slide,
                               color=clust_col ,
                               img_key='hires', 
                               labels=clust_labels,
        show_img=True,
        # 'fast' (white background) or 'dark_background'
        style='dark_background',
        # limit color scale at 99.2% quantile of cell abundance
        max_color_quantile=0.992,
        # size of locations (adjust depending on figure size)
        circle_diameter=4,
        colorbar_position='right')

In [ ]:
clust_col = {
    "Fibroblasts": "purple",  # 替换为您想要的颜色
    "Myeloid": "yellow",
    # 继续为其他群集指定颜色...
    'T/ILC':"red",
    'B/Plasma':'green'
}

In [ ]:
with mpl.rc_context({'figure.figsize': (15, 15)}):
    fig = plot_spatial(
        adata=slide,
        # labels to show on a plot
        color=clust_col,  # 您的细胞类型列表
        #reorder_cmap=reorder_cmap,
        labels=clust_labels,
        show_img=True,
        # 'fast' (white background) or 'dark_background'
        style='dark_background',
        # limit color scale at 99.2% quantile of cell abundance
        max_color_quantile=0.992,
        # size of locations (adjust depending on figure size)
        circle_diameter=6,
        colorbar_position='right'
    )

In [ ]:
# Compute expected expression per cell type
expected_dict = mod.module.model.compute_expected_per_cell_type(
    mod.samples["post_sample_q05"], mod.adata_manager
)

# Add to anndata layers
for i, n in enumerate(mod.factor_names_):
    adata_vis.layers[n] = expected_dict['mu'][i]

# Save anndata object with results
adata_file = f"{run_name}/sp.h5ad"
adata_vis.write(adata_file)
adata_file

In [ ]:
# list cell types and genes for plotting
ctypes = ['B/Plasma', 'Endothelial', 'Epithelial', 'Fibroblasts', 'Myeloid', 'T/ILC']
genes = ['CD3D', 'CR2']

with mpl.rc_context({'axes.facecolor':  'black'}):
    # select one slide
    slide = select_slide(adata_vis, 'gan1')
    slide.obsm['spatial'] = np.array(slide.obsm['spatial'], dtype=float)
    from tutorial_utils import plot_genes_per_cell_type
    plot_genes_per_cell_type(slide, genes, ctypes);